In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import calendar as cal

# read the file
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/67cd5b7b956aa6979116b645539df874f6f79d18cdce8315e08117f8.csv')

# parse the date
df['Date'] = pd.to_datetime(df['Date'])

# add group column the year values
df['Day'] =   df['Date'].map(lambda x: '%02d' % x.month + '-'+ '%02d' % x.day)
#df['Month'] = df['Date'].map(lambda x: cal.month_name[x.month])

# build 2 data sets and remove Feb 29
plot_data_2014 = df[(df['Date'] >= '2008-01-01') & (df['Date'] < '2015-01-01') & (df['Day'] != '02-29')]
plot_data_2015 = df[(df['Date'] >= '2015-01-01') & (df['Date'] < '2016-01-01') & (df['Day'] != '02-29')]

# reduce columns
columns_to_keep = ['Day', 'Data_Value']
plot_data_2014=plot_data_2014[columns_to_keep]
columns_to_keep = ['Day', 'Data_Value','Date']
plot_data_2015=plot_data_2015[columns_to_keep]

# find extrema
plot_data_2014 = plot_data_2014.groupby('Day')['Data_Value'].agg({'max2014': "max", 'min2014': "min"})
plot_data_2015 = plot_data_2015.groupby(['Day','Date'])['Data_Value'].agg({'max2015': "max", 'min2015': "min"})

#merge dfs and filter out where 2015 > 2014max or 2015 < 2014min
merger = pd.merge(plot_data_2015,plot_data_2014,how='inner', left_index=True, right_index=True)
merger['high'] = np.where(merger['max2015'] > merger['max2014'], merger['max2015'],np.nan)
merger['low'] = np.where(merger['min2015'] < merger['min2014'],  merger['min2015'],np.nan)
merger.reset_index(level=1, inplace=True)

merger['low'] = merger['low']/10.0
merger['high'] = merger['high']/10.0
merger['min2014'] = merger['min2014']/10.0
merger['max2014'] = merger['max2014']/10.0

# generate lists
#date_set = merger['Date'].tolist()
min_set = merger['min2014'].tolist()
max_set = merger['max2014'].tolist()
low_set = merger['low'].tolist()
high_set = merger['high'].tolist()

plt.figure()

plt.plot(min_set, 'b-', alpha=0.4)
plt.plot( max_set, 'r-', alpha=0.4)
low, = plt.plot( low_set, 'bo')
high, = plt.plot( high_set, 'ro')

# make nice
plt.xlabel('Day of Year')
plt.ylabel('Temperature (°C)')
plt.title('2015 Temperature Records vs. previous 10 year ranges per day \n Munich Germany',fontweight="bold")

labels = ['Jan', 'Feb', 'Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
plt.xticks( [i*30.4 for i in range(12)], labels)

ax = plt.gca()
ttl = ax.title
ttl.set_position([.5, 1.05])
ax.xaxis.labelpad = 10

plt.tick_params(top='off', bottom='off', left='on', right='off', labelleft='on', labelbottom='on')

plt.gca().fill_between(range(len(min_set)), 
                       min_set, max_set, 
                       facecolor='blue', 
                       alpha=0.25)

for spine in plt.gca().spines.values():
    spine.set_visible(False)

plt.legend([high, low], ['2015 set new record high', '2015 set new record low'], loc='lower center',frameon=False)
   
# resize
fig = plt.gcf()
fig.set_size_inches(10, 6)
fig.savefig('munich.png', dpi=100)

plt.show()

